In [386]:
import pandas as pd
import numpy as np
import re
import os
import folium

### PD Set Options

In [387]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 800)

## Read in data

In [388]:
df = pd.read_csv('redfin_2023-02-09-13-03-46.csv')

In [389]:
df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,February-9-2023,Condo/Co-op,19316 NE 26th Ave Unit 184C,Miami,FL,33180.0,385000.0,2.0,2.0,Riverwood Sec 3,1579.0,NaN,1973.0,NaN,244.0,714.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19316-Ne-26th-Ave-33180/unit-184C/home/43007516,Beaches MLS,F10356567,N,Y,25.954511,-80.151217
1,PAST SALE,February-9-2023,Condo/Co-op,1000 Quayside Ter #912,Miami,FL,33138.0,382000.0,2.0,2.0,TOWERS OF QUAYSIDE CONDO,1538.0,NaN,1981.0,NaN,248.0,1851.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Quayside-Ter-33138/unit-912/home/43018175,MARMLS,A11231127,N,Y,25.873714,-80.168380
2,PAST SALE,February-9-2023,Condo/Co-op,465 Brickell Ave #3104,Miami,FL,33131.0,460000.0,1.0,1.0,ICONBRICKELL CONDO NO 1,597.0,NaN,2008.0,NaN,771.0,756.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/465-Brickell-Ave-33131/unit-3104/home/43449538,MARMLS,A11311107,N,Y,25.768852,-80.188538
3,PAST SALE,February-8-2023,Condo/Co-op,10790 N Kendall Dr Unit C16,Miami,FL,33176.0,200000.0,1.0,1.0,SPANISH TRACE CONDO,630.0,NaN,1968.0,NaN,317.0,238.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10790-N-Kendall-Dr-33176/unit-C16/home/43145131,MARMLS,A11308504,N,Y,25.685254,-80.367568
4,PAST SALE,February-8-2023,Condo/Co-op,888 S Douglas Rd #1101,Coral Gables,FL,33134.0,545000.0,2.0,2.0,PUERTA DE PALMAS CONDO,1072.0,NaN,2007.0,NaN,508.0,1032.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/888-S-Douglas-Rd-33134/unit-1101/home/43378704,MARMLS,A11302343,N,Y,25.763257,-80.255704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,PAST SALE,November-11-2022,Condo/Co-op,130 SW 108th Ave Unit J1,Sweetwater,FL,33174.0,330000.0,3.0,2.0,WEST FLAGLER HERITAGE NO,1044.0,NaN,1975.0,NaN,316.0,300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/130-SW-108th-Ave-33174/unit-J1/home/42921741,MARMLS,A11231263,N,Y,25.766814,-80.371048
2746,PAST SALE,November-11-2022,Condo/Co-op,1111 NW 19th Ave #103,Miami,FL,33125.0,205000.0,2.0,1.5,LA BAHIA CONDO,758.0,NaN,1974.0,NaN,270.0,250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1111-NW-19th-Ave-33125/unit-103/home/42720557,MARMLS,A11253827,N,Y,25.784089,-80.226322
2747,PAST SALE,November-11-2022,Condo/Co-op,10201 E Bay Harbor Dr #606,Bay Harbor Islands,FL,33154.0,1400000.0,1.0,1.5,SERENO RESIDENCES CONDO,1029.0,NaN,2016.0,NaN,1361.0,1173.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/10201-E-Bay-Harbor-Dr-33154/unit-606/home/112894046,MARMLS,A11300720,N,Y,25.892681,-80.130898
2748,PAST SALE,November-11-2022,Condo/Co-op,18001 N Bay Rd #309,Sunny Isles Beach,FL,33160.0,350000.0,2.0,2.0,ATRIUM GARDENS CONDO,970.0,NaN,1981.0,NaN,361.0,362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18001-N-Bay-Rd-33160/unit-309/home/43238892,MARMLS,A11286320,N,Y,25.943756,-80.124058


## Data validation

In [390]:
### Remove NaNs from 'SOLD DATE' ###
df['SOLD DATE'] = df['SOLD DATE'].fillna('Not provided')

In [391]:
### Filter to just the last month's data ###
x = df['SOLD DATE'].str.startswith('January')
df = df[x]

In [392]:
### Ensure that only 'PROPERTY TYPE' Condo/Co-op is in the data ###
df = df.loc[df['PROPERTY TYPE'] == 'Condo/Co-op']

In [393]:
### Sort properties by sale price, with highest sale price at the top ###
df = df.sort_values(by='PRICE',ascending=False)
### Create a price column formatted as currency ###
df['PRICE_AS_CURRENCY'] = df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df['BEDS'] = df['BEDS'].apply(lambda x: '{:,.0f}'.format(x))
df['BATHS'] = df['BATHS'].apply(lambda x: '{:,.0f}'.format(x))
df['YEAR BUILT'] = df['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df['PRICE_SQUARE_FEET_AS_CURRENCY'] = df['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [394]:
### Insert different colors for top 10 sales vs. the rest ###
df['COLOR'] = ''
### Create RANK column ###
df['RANK'] = 0
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [395]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [396]:
### Drop the columns ###
df = df.drop(columns=columns_drop)

In [397]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

## Make Map

In [398]:
### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10.5,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"January 2023 Condo Sales")
m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [399]:
m.save('index.html')

## Summary Info

In [400]:
print('SALES INFO')
print(f'Number of sales: {len(df)}')
print('--------')
print(f'Total sale price: ${df["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df["PRICE"].min():,.0f}')
print('--------')
print(f'Max sale price: ${df["PRICE"].max():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest buidling: {df["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df["YEAR BUILT"].mean()}')
print('------------------------------------------------')
print('BEDS & BATHS')
print(f'Most beds: {df["BEDS"].max()}')
print('----------')
print(f'Fewest beds: {df["BEDS"].min()}')
print('----------')
print(f'Most baths: {df["BATHS"].max()}')
print('----------')
print(f'Fewest baths: {df["BATHS"].min()}')

SALES INFO
Number of sales: 637
--------
Total sale price: $442,245,084
--------
Median sale price: $398,000
--------
Max sale price: $17,000,000
--------
Min sale price: $105,000
--------
Max sale price: $17,000,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,161
--------
Min price per square foot: $127
--------
Median price per square foot: $418
------------------------------------------------
CONDO AGES
Newest building: 2022
----------
Oldest buidling: 1925
----------
Average building age: inf
------------------------------------------------
BEDS & BATHS
Most beds: 6
----------
Fewest beds: 0
----------
Most baths: 8
----------
Fewest baths: 1


In [401]:
df.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'PRICE_SQUARE_FEET_AS_CURRENCY',
       'COLOR', 'RANK'],
      dtype='object')

Most expensive
Continuum South Tower, 100 South Pointe Drive, unit 1206 | 95 days on the market | $10.3M | $4,087 psf | Listing agents: Bill Hernandez and Bryan Sereny, both with Douglas Elliman | Buyer’s agent: Matthew Wilkens with One Sotheby’s International Realty


In [402]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [403]:
### Convert 'YEAR BUILT' back to integer ###
df['YEAR BUILT'] = pd.to_numeric(df['YEAR BUILT'])

In [404]:
print(f"{ME}{BR}{df.loc[df['PRICE'].idxmax()]['LOCATION']}, {df.loc[df['PRICE'].idxmax()]['ADDRESS']} | Price ${df.loc[df['PRICE'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['PRICE'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['PRICE'].idxmax()]['YEAR BUILT']}")
print(f"{LE}{BR}{df.loc[df['PRICE'].idxmin()]['LOCATION']}, {df.loc[df['PRICE'].idxmin()]['ADDRESS']} | Price ${df.loc[df['PRICE'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['PRICE'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['PRICE'].idxmin()]['YEAR BUILT']}")

print(f"{MAX_PSF}{BR}{df.loc[df['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df.loc[df['$/SQUARE FEET'].idxmax()]['ADDRESS']} | Price ${df.loc[df['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['$/SQUARE FEET'].idxmax()]['YEAR BUILT']}")
print(f"{MIN_PSF}{BR}{df.loc[df['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df.loc[df['$/SQUARE FEET'].idxmin()]['ADDRESS']} | Price ${df.loc[df['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['$/SQUARE FEET'].idxmin()]['YEAR BUILT']}")

print(f"{Newest}{BR}{df.loc[df['YEAR BUILT'].idxmax()]['LOCATION']}, {df.loc[df['YEAR BUILT'].idxmax()]['ADDRESS']} | Price ${df.loc[df['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['YEAR BUILT'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['YEAR BUILT'].idxmin()]['YEAR BUILT']}")
print(f"{Oldest}{BR}{df.loc[df['YEAR BUILT'].idxmin()]['LOCATION']}, {df.loc[df['YEAR BUILT'].idxmin()]['ADDRESS']} | Price ${df.loc[df['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['YEAR BUILT'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['YEAR BUILT'].idxmin()]['YEAR BUILT']}")

Most Expensive
Arte Surfside, 8955 Collins Ave #201 | Price $17,000,000 | $2437.0 psf | Year built: 2019
Least Expensive
THE BLUE A RESORT HOTEL CO, 5300 NW 87th Ave #1306 | Price $105,000 | $149.0 psf | Year built: 2008
Highest Price Per Square Foot
FISHER ISLAND, 6800 Fisher Island Dr #6823 | Price $15,500,000 | $3161.0 psf | Year built: 2019
Lowest Price Per Square Foot
BUCKLEY TOWERS CONDO - EA, 1351 NE Miami Gardens Dr Unit 1125E | Price $145,000 | $127.0 psf | Year built: 1969
Newest
FUSION AT THE GROVE CONDO, 2841 SW 27 St Unit A | Price $850,000 | $nan psf | Year built: 1925
Oldest
ARVILLA CONDO, 1117 Euclid Ave #102 | Price $380,000 | $495.0 psf | Year built: 1925


## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [405]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [406]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/January_2022_monthly_condo_sales
